In [1]:
# Change all of these variables according to application

# Path to the trained model
model_path = r"E:\Perk\Data\SavedModels\1-UBC\UltrasoundBoneSegmentation-Train-TF2_model-0_2020-08-07_19-58-51.h5"
# Path to testing images
image_folder_path = r"E:\Perk\Dataset\Sub-evaluation-1-UBC\Sub-evaluation-1-UBC\test\images"
# Folder to save output predictions in
output_folder = r"E:\Perk\Data\Output\Post-threshold-second-try\Sub-evaluation-1-UBC"
# Optimal threshold for predictions, as determined by the training notebook
best_thresh = 0.55
# Notebook can only predict on 2d png, or 3d nrrd
three_dim_input = False
# Only used for the 2d predictions, in order to determine the best threshold
output_as_three_dim = False

In [2]:
import os
import sys
from pathlib import Path
from PIL import Image
import cv2
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import evaluation_metrics

import nrrd

In [3]:
# Import aigt modules

parent_folder = os.path.dirname(os.path.abspath(os.curdir))
sys.path.append(parent_folder)

import Models.segmentation_unet as unet
import utils

In [4]:
from tensorflow.keras.models import load_model

model = load_model(model_path,custom_objects={'loss': unet.weighted_categorical_crossentropy([0.1,0.9])})

In [5]:
def predict_2d(image):
    image = convert_to_1channel(image)
    return model.predict(image)

In [6]:
def preprocess_data(input_arr,dsize,binarize):
    arr = cv2.resize(input_arr, dsize=dsize, interpolation=cv2.INTER_AREA)
    arr=arr/255
    if binarize == True:
        arr=arr>0
        
    arr = arr[:,:,np.newaxis]
    arr = arr[np.newaxis,:,:,:]
    
    return arr

In [8]:
filename_array,final_pred_array = [],[]

if three_dim_input == False:
    image_dimensions,image_array = [],[]
    for root, dirs, files in os.walk(image_folder_path):
        for file in files:
            if file.endswith('.png'):
                image_path=os.path.join(root, file)
                filename_array.append(file)
                arr=np.array(cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2GRAY)).astype(np.uint8)
                image_dimensions.append(np.shape(arr))

                image_array.append(preprocess_data(np.array(arr),(128,128),False))
    
    n_images = np.shape(image_array)[0]
    for image in range(n_images):
        pred = (model.predict(image_array[image]))[0][:,:,1]
        final_pred_array.append(pred)
        
        if output_as_three_dim == False:
            save_path = os.path.join(output_folder,filename_array[image])
            pred = cv2.resize(pred, dsize=(image_dimensions[image][1],image_dimensions[image][0]))
            thresh_pred = (pred>best_thresh)
            
            cv2.imwrite(save_path, (thresh_pred.astype(np.uint8))*255) 
    
    print(np.shape(final_pred_array))
    if output_as_three_dim == True:
        plt.imshow(final_pred_array[0])
        print("3d")
        save_path = os.path.join(output_folder,(filename_array[image])[:-4]+".nrrd")
        nrrd.write(save_path,((np.array(final_pred_array))))

(3308, 128, 128)


In [ ]:
if three_dim_input == True:
    for root, dirs, files in os.walk(image_folder_path):
        for file in files:
            if file.endswith('.nrrd'):
                
                final_pred_array = []
                                
                image_dimensions_along_x = []
                x_axis_images = []
                
                image_path=os.path.join(root, file)
                filename_array.append(file)

                data, header = nrrd.read(image_path)
                print(np.shape(data))
                
                xDim,yDim,zDim = data.shape[0],data.shape[1],data.shape[2]
                image_dimensions_along_x = np.shape(data[0,:,:])

                # The x axis was chosen as the testing axis arbitrarily
                # This is because the testing data does not tend to be oriented the same way across scans
                for image in range(xDim):
                    x_axis_images.append(preprocess_data(np.array(data[image,:,:]),(128,128),False))
                    
                    # change this order once testing is done
                    pred = (model.predict(x_axis_images[image])[0][:,:,1])
                    pred = cv2.resize(pred, dsize=(zDim,yDim))
                    
                    thresh_pred = pred>best_thresh
                    final_pred_array.append(thresh_pred)
                    
                save_path = os.path.join(output_folder,file)
                
                print(np.shape(final_pred_array))
                nrrd.write(save_path,(np.array(final_pred_array)).astype(int),header)
                
                print("Saved prediction for {}".format(file))

In [ ]:
#TODO generate diagram of test predictions